In [1]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch
import evaluate
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset, DatasetDict
from collections import Counter
from torch.utils.data.sampler import WeightedRandomSampler
from transformers import get_scheduler
from tqdm.notebook import tqdm
import os
from utils.get_embeddings import load_df

Попробуем просто обучить выход берта

In [2]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Заменим метки классов на 0 и 1

In [3]:
df = load_df(os.path.join('data', 'train_spam.csv'))
df['text_type'] = df['text_type'].apply(lambda x: {'ham': 0, 'spam': 1}[x])
df

,text_type,text
0,0,make sure alex knows his birthday is over in f...
1,0,a resume for john lavorato thanks vince i will...
2,1,plzz visit my website moviesgodml to get all m...
3,1,urgent your mobile number has been awarded wit...
4,0,overview of hr associates analyst project per ...
...,...,...
16273,1,if you are interested in binary options tradin...
16274,1,dirty pictureblyk on aircel thanks you for bei...
16275,0,or you could do this g on mon 1635465 sep 1635...
16276,0,insta reels par 80 गंद bhara pada hai 👀 kuch b...


# Преобразуем в формат dataset

In [4]:
def df2dict(df):
    new_dict = {'text': df['text'].to_list(), 'label': df['text_type'].to_list()}
    return new_dict

In [5]:
df_train, df_test = train_test_split(df, test_size=0.25)

In [6]:
train_test_dict = {'train': Dataset.from_dict(df2dict(df_train)),
                   'test': Dataset.from_dict(df2dict(df_test))}

In [7]:
dataset = DatasetDict(train_test_dict)

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 12208
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 4070
    })
})

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt")

In [10]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/12208 [00:00<?, ? examples/s]

Map:   0%|          | 0/4070 [00:00<?, ? examples/s]

In [11]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

In [12]:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [13]:
tokenized_datasets.set_format("torch")

# Обучим модель

In [14]:
c = Counter(i.item() for i in tokenized_datasets['train']['labels'])
c

Counter({0: 8595, 1: 3613})

Вообще говоря виден дисбаланс классов, но с сэмплером по итогу берт обучился хуже

In [15]:
weights = [1/c.get(i.item()) for i in tokenized_datasets['train']['labels']]
sampler = WeightedRandomSampler(weights, num_samples=len(weights))

In [16]:
train_dataloader = DataLoader(tokenized_datasets["train"], batch_size=32, shuffle=True)
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=32)

In [17]:
optimizer = Adam(model.parameters(), lr=5e-4)

In [18]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [19]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

Здесь я заморожу веса в берте. Вообще говоря, если дообучать все слои, то это намного дольше, плюс требует много памяти, так что батчи будут маленькие. В целом я пробовал дообучать все слои с батчем 8, но результат был похуже

In [20]:
for param in model.bert.parameters():
    param.requires_grad = False

In [21]:
model.train()
for epoch in tqdm(range(num_epochs)):
    for batch in tqdm(train_dataloader, leave=False):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/382 [00:00<?, ?it/s]

  0%|          | 0/382 [00:00<?, ?it/s]

  0%|          | 0/382 [00:00<?, ?it/s]

In [22]:
metric = evaluate.load("roc_auc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.min(torch.sigmoid(logits), dim=1).values
    metric.add_batch(prediction_scores=predictions, references=batch["labels"])

metric.compute()

{'roc_auc': 0.8724366538426628}

In [23]:
metric = evaluate.combine(["accuracy", "recall", "precision", "f1"])

model.eval()

for batch in eval_dataloader:

    batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():

        outputs = model(**batch)

    logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1)

    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8049140049140049,
 'recall': 0.36705685618729095,
 'precision': 0.9222689075630253,
 'f1': 0.5251196172248804}

Видим высокую точность, но низкую полноту, что говорит о том, что модель если модель предсказывает положительный класс на каком-то объекте, то он скорее всего будет действительно положительного класса, но вот определяет она объекты положительного класса среди всех довольно плохо.